In [15]:
# !pip install censusgeocode

In [42]:
import glob
import json
import requests
import pandas as pd
from pprint import pprint

# Census Examples 

This notebook uses the `censusgeocode` package in Python (which is simply a wrapper around the US Census' official Geocoder API) to get census geographies for list of addresses or lat/longs

- https://pypi.org/project/censusgeocode/

### Step 1 | Grab your data at the address level

In [43]:
df = pd.read_csv('plants_with_coordinates.csv')

In [44]:
df.head()

,Unnamed: 0,plant_name,Plant Code,State,Sector Name,Prime Movers,Fuel Types,Primary Technology,withdrawal_volume_million_gallons,discharge_volume_million_gallons,consumption_volume_million_gallons,energy_category,map_link,longitude,latitude
0,0,(3K) 59 Hetcheltown Rd,66729,NY,IPP Non-CHP,PV,SUN,Solar Photovoltaic,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-73.91048,42.87657&level=14",-73.910480,42.876570
1,1,0 Hammond St CSG,64876,MA,IPP Non-CHP,"BA, PV","MWH, SUN",Multiple,NaN,NaN,NaN,Other,"map/?center=-70.726675,41.808547&level=14",-70.726675,41.808547
2,2,1 Commercial,67464,MA,IPP Non-CHP,"BA, PV","MWH, SUN",Multiple,NaN,NaN,NaN,Other,"map/?center=-71.237,42.115&level=14",-71.237000,42.115000
3,3,"10 Briggs Solar NG, LLC (East)",62781,RI,IPP Non-CHP,PV,SUN,Solar Photovoltaic,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-71.49625,41.63269&level=14",-71.496250,41.632690
4,4,"10 Finderne Avenue Solar, LLC",64023,NJ,IPP Non-CHP,PV,SUN,Solar Photovoltaic,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-74.57594,40.55812&level=14",-74.575940,40.558120


In [45]:
df['plant_name'].nunique()

12633

In [46]:
df.shape

(12661, 15)

In [37]:
df.to_csv('plants_with_coordinates.csv',index=False)

### Step 2 | Geoode Lat/Long if they're not already present

It already exists in this dataset. Census geocode has a function to go from addresss --> lat/long, but I haven't had time to implement it here. This dataset already has lat/longs. Message me if you're struggling with this step.

In [47]:
import pandas as pd
import requests_cache
import time
from tqdm import tqdm

# Enable caching to speed up repeated requests
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng, retries=3, delay=1):
    url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
    params = {
        "x": lng,
        "y": lat,
        "benchmark": "Public_AR_Census2020",
        "vintage": "Census2020_Census2020",
        "format": "json"
    }
    for attempt in range(retries):
        try:
            response = cache.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            tract = data['result']['geographies']['Census Tracts'][0]
            return tract
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(delay * (attempt + 1))  # Exponential backoff
            else:
                return {"error": str(e), "lat": lat, "lng": lng}

def geocode_all(df, save_path="census_geos_all.csv"):
    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing all data"):
        lat, lng = row['latitude'], row['longitude']
        result = geocode(lat, lng)
        results.append(result)

    # Convert the results into a DataFrame
    df_results = pd.DataFrame(results)
    df_results.to_csv(save_path, index=False)

    # Free memory by deleting intermediate data
    del results, df_results  
    print(f"Results saved to {save_path}")

# Now, run this on your DataFrame
geocode_all(df)


Processing all data: 100%|███████████████| 12661/12661 [00:25<00:00, 488.71it/s]


Results saved to census_geos_all.csv


In [48]:
# import pandas as pd
# import requests_cache
# import time
# import os
# from tqdm import tqdm

# # Enable caching to speed up repeated requests
# cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

# def geocode(lat, lng, retries=3, delay=1):
#     url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
#     params = {
#         "x": lng,
#         "y": lat,
#         "benchmark": "Public_AR_Census2020",
#         "vintage": "Census2020_Census2020",
#         "format": "json"
#     }
#     for attempt in range(retries):
#         try:
#             response = cache.get(url, params=params, timeout=10)
#             response.raise_for_status()
#             data = response.json()
#             tract = data['result']['geographies']['Census Tracts'][0]
#             return tract
#         except Exception as e:
#             if attempt < retries - 1:
#                 time.sleep(delay * (attempt + 1))  # Exponential backoff
#             else:
#                 return {"error": str(e), "lat": lat, "lng": lng}

# def process_chunk(chunk, chunk_index, save_prefix="census_geos_part"):
#     results = []
#     for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Processing chunk {chunk_index + 1}"):
#         lat, lng = row['latitude'], row['longitude']
#         result = geocode(lat, lng)
#         results.append(result)

#     # Convert the results into a DataFrame
#     df_chunk = pd.DataFrame(results)
#     output_path = f"{save_prefix}_{chunk_index + 1}.csv"
#     df_chunk.to_csv(output_path, index=False)

#     # Free memory by deleting intermediate data
#     del results, df_chunk  
#     return output_path

# def geocode_in_chunks_safe(df, chunk_size=500, save_prefix="census_geos_part"):
#     total_chunks = (len(df) + chunk_size - 1) // chunk_size  # Calculate number of chunks
    
#     for i in range(total_chunks):
#         output_file = f"{save_prefix}_{i + 1}.csv"
#         print(f"Processing chunk {i + 1} of {total_chunks}")
        
#         # Skip checking if file exists; process each chunk regardless
#         chunk = df.iloc[i*chunk_size : (i+1)*chunk_size]  # Select the chunk
#         process_chunk(chunk, i, save_prefix=save_prefix)
#         time.sleep(2)  # Sleep between requests to avoid hitting API rate limits

# # Now, run this on your DataFrame
# geocode_in_chunks_safe(df)


In [49]:
# import glob

# # Adjust this pattern to match your saved files
# chunk_files = sorted(glob.glob("census_geos_part_*.csv"))

# # Load and concatenate them
# combined_df = pd.concat([pd.read_csv(f) for f in chunk_files], ignore_index=True)

# # Optional: Save combined result to disk
# combined_df.to_csv("census_geocoded_full.csv", index=False)

# # Check it out
# combined_df.head()


In [50]:
# combined_df.shape

In [51]:
census_geos = pd.read_csv('census_geos_all.csv')

In [52]:
census_geos

,POP100,GEOID,CENTLAT,AREAWATER,STATE,BASENAME,OID,LSADC,FUNCSTAT,INTPTLAT,...,CENTLON,HU100,AREALAND,INTPTLON,MTFCC,UR,COUNTY,error,lat,lng
0,3494.0,3.609303e+10,42.881043,69888.0,36.0,325.02,2.079074e+13,CT,S,42.881277,...,-73.913102,1320.0,7.206301e+06,-73.911677,G5020,M,93.0,NaN,NaN,NaN
1,6075.0,2.502354e+10,41.857932,4840087.0,25.0,5442.00,2.079021e+13,CT,S,41.858697,...,-70.744259,2689.0,6.948583e+07,-70.745874,G5020,M,23.0,NaN,NaN,NaN
2,5862.0,2.502141e+10,42.119699,338878.0,25.0,4141.00,2.079026e+13,CT,S,42.120093,...,-71.205044,2145.0,2.839787e+07,-71.203693,G5020,M,21.0,NaN,NaN,NaN
3,6876.0,4.400302e+10,41.633007,129489.0,44.0,209.03,2.079023e+13,CT,S,41.633184,...,-71.517378,2301.0,3.166488e+07,-71.524229,G5020,M,3.0,NaN,NaN,NaN
4,7319.0,3.403505e+10,40.562949,620006.0,34.0,510.00,2.079031e+13,CT,S,40.563551,...,-74.571689,2702.0,1.127632e+07,-74.570409,G5020,U,35.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12656,1836.0,8.087001e+09,40.152142,314014.0,8.0,8.00,2.079036e+13,CT,S,40.154192,...,-103.615312,793.0,8.739758e+08,-103.651230,G5020,M,87.0,NaN,NaN,NaN
12657,1900.0,1.326995e+10,32.499910,1792585.0,13.0,9503.01,2.079015e+15,CT,S,32.485892,...,-84.326620,1031.0,3.932802e+08,-84.327847,G5020,R,269.0,NaN,NaN,NaN
12658,3746.0,4.204302e+10,40.257994,2287145.0,42.0,201.00,2.079047e+13,CT,S,40.257558,...,-76.884192,2764.0,2.290375e+06,-76.881726,G5020,U,43.0,NaN,NaN,NaN
12659,1134.0,1.210502e+10,27.746262,80407214.0,12.0,161.00,2.079037e+13,CT,S,27.745906,...,-81.973661,482.0,2.774396e+08,-81.978291,G5020,R,105.0,NaN,NaN,NaN


In [53]:
census_geos['GEOID'] = census_geos['GEOID'].apply(lambda x: f"{int(x):011d}" if pd.notna(x) else x)

In [54]:
census_geos

,POP100,GEOID,CENTLAT,AREAWATER,STATE,BASENAME,OID,LSADC,FUNCSTAT,INTPTLAT,...,CENTLON,HU100,AREALAND,INTPTLON,MTFCC,UR,COUNTY,error,lat,lng
0,3494.0,36093032502,42.881043,69888.0,36.0,325.02,2.079074e+13,CT,S,42.881277,...,-73.913102,1320.0,7.206301e+06,-73.911677,G5020,M,93.0,NaN,NaN,NaN
1,6075.0,25023544200,41.857932,4840087.0,25.0,5442.00,2.079021e+13,CT,S,41.858697,...,-70.744259,2689.0,6.948583e+07,-70.745874,G5020,M,23.0,NaN,NaN,NaN
2,5862.0,25021414100,42.119699,338878.0,25.0,4141.00,2.079026e+13,CT,S,42.120093,...,-71.205044,2145.0,2.839787e+07,-71.203693,G5020,M,21.0,NaN,NaN,NaN
3,6876.0,44003020903,41.633007,129489.0,44.0,209.03,2.079023e+13,CT,S,41.633184,...,-71.517378,2301.0,3.166488e+07,-71.524229,G5020,M,3.0,NaN,NaN,NaN
4,7319.0,34035051000,40.562949,620006.0,34.0,510.00,2.079031e+13,CT,S,40.563551,...,-74.571689,2702.0,1.127632e+07,-74.570409,G5020,U,35.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12656,1836.0,08087000800,40.152142,314014.0,8.0,8.00,2.079036e+13,CT,S,40.154192,...,-103.615312,793.0,8.739758e+08,-103.651230,G5020,M,87.0,NaN,NaN,NaN
12657,1900.0,13269950301,32.499910,1792585.0,13.0,9503.01,2.079015e+15,CT,S,32.485892,...,-84.326620,1031.0,3.932802e+08,-84.327847,G5020,R,269.0,NaN,NaN,NaN
12658,3746.0,42043020100,40.257994,2287145.0,42.0,201.00,2.079047e+13,CT,S,40.257558,...,-76.884192,2764.0,2.290375e+06,-76.881726,G5020,U,43.0,NaN,NaN,NaN
12659,1134.0,12105016100,27.746262,80407214.0,12.0,161.00,2.079037e+13,CT,S,27.745906,...,-81.973661,482.0,2.774396e+08,-81.978291,G5020,R,105.0,NaN,NaN,NaN


In [55]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT']
census_geos_df = census_geos[to_keep]
census_geos_df

,GEOID,STATE,COUNTY,TRACT
0,36093032502,36.0,93.0,32502.0
1,25023544200,25.0,23.0,544200.0
2,25021414100,25.0,21.0,414100.0
3,44003020903,44.0,3.0,20903.0
4,34035051000,34.0,35.0,51000.0
...,...,...,...,...
12656,08087000800,8.0,87.0,800.0
12657,13269950301,13.0,269.0,950301.0
12658,42043020100,42.0,43.0,20100.0
12659,12105016100,12.0,105.0,16100.0


In [56]:
df_with_geos = pd.concat(
    [ 
        df.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

df_with_geos.head()

,Unnamed: 0,plant_name,Plant Code,State,Sector Name,Prime Movers,Fuel Types,Primary Technology,withdrawal_volume_million_gallons,discharge_volume_million_gallons,consumption_volume_million_gallons,energy_category,map_link,longitude,latitude,GEOID,STATE,COUNTY,TRACT
0,0,(3K) 59 Hetcheltown Rd,66729,NY,IPP Non-CHP,PV,SUN,Solar Photovoltaic,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-73.91048,42.87657&level=14",-73.910480,42.876570,36093032502,36.0,93.0,32502.0
1,1,0 Hammond St CSG,64876,MA,IPP Non-CHP,"BA, PV","MWH, SUN",Multiple,NaN,NaN,NaN,Other,"map/?center=-70.726675,41.808547&level=14",-70.726675,41.808547,25023544200,25.0,23.0,544200.0
2,2,1 Commercial,67464,MA,IPP Non-CHP,"BA, PV","MWH, SUN",Multiple,NaN,NaN,NaN,Other,"map/?center=-71.237,42.115&level=14",-71.237000,42.115000,25021414100,25.0,21.0,414100.0
3,3,"10 Briggs Solar NG, LLC (East)",62781,RI,IPP Non-CHP,PV,SUN,Solar Photovoltaic,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-71.49625,41.63269&level=14",-71.496250,41.632690,44003020903,44.0,3.0,20903.0
4,4,"10 Finderne Avenue Solar, LLC",64023,NJ,IPP Non-CHP,PV,SUN,Solar Photovoltaic,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-74.57594,40.55812&level=14",-74.575940,40.558120,34035051000,34.0,35.0,51000.0


In [57]:
df_with_geos.tail()

,Unnamed: 0,plant_name,Plant Code,State,Sector Name,Prime Movers,Fuel Types,Primary Technology,withdrawal_volume_million_gallons,discharge_volume_million_gallons,consumption_volume_million_gallons,energy_category,map_link,longitude,latitude,GEOID,STATE,COUNTY,TRACT
12656,12656,Pawnee,6248,CO,Electric Utility,ST,SUB,Conventional Steam Coal,1970,0,1970,Coal,"map/?center=-103.6803,40.2217&level=14",-103.680300,40.221700,08087000800,8.0,87.0,800.0
12657,12657,Pawpaw Solar Plant,59894,GA,IPP Non-CHP,PV,SUN,Solar Photovoltaic,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-84.256778,32.572875&level=14",-84.256778,32.572875,13269950301,13.0,269.0,950301.0
12658,12658,Paxton Creek Cogeneration,50373,PA,IPP CHP,IC,NG,Natural Gas Internal Combustion Engine,NaN,NaN,NaN,Natural Gas,"map/?center=-76.8771,40.2653&level=14",-76.877100,40.265300,42043020100,42.0,43.0,20100.0
12659,12659,Payne Creek Solar,61665,FL,Electric Utility,PV,SUN,Solar Photovoltaic,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-81.964222,27.664583&level=14",-81.964222,27.664583,12105016100,12.0,105.0,16100.0
12660,12660,Payne's Ferry,57124,ID,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-115.010833,42.825278&level=14",-115.010833,42.825278,16083001600,16.0,83.0,1600.0


In [58]:
df_with_geos.shape

(12661, 19)

In [59]:
df_with_geos[df_with_geos['GEOID'] == '06029006012']

,Unnamed: 0,plant_name,Plant Code,State,Sector Name,Prime Movers,Fuel Types,Primary Technology,withdrawal_volume_million_gallons,discharge_volume_million_gallons,consumption_volume_million_gallons,energy_category,map_link,longitude,latitude,GEOID,STATE,COUNTY,TRACT
90,90,85 A,52162,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.324772,35.095797&level=14",-118.324772,35.095797,06029006012,6.0,29.0,6012.0
91,91,85 B,52163,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.332533,35.102577&level=14",-118.332533,35.102577,06029006012,6.0,29.0,6012.0
378,378,Alta Wind Energy Center I,57282,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.353422,35.033725&level=14",-118.353422,35.033725,06029006012,6.0,29.0,6012.0
379,379,Alta Wind Energy Center II,57291,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.295278,35.023889&level=14",-118.295278,35.023889,06029006012,6.0,29.0,6012.0
380,380,Alta Wind Energy Center III,57292,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.295278,35.023889&level=14",-118.295278,35.023889,06029006012,6.0,29.0,6012.0
385,385,Alta Wind XI,58395,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.39,35.065&level=14",-118.390000,35.065000,06029006012,6.0,29.0,6012.0
1797,1797,CTV Power Purchase Contract Trust,54300,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.3083,35.0583&level=14",-118.308300,35.058300,06029006012,6.0,29.0,6012.0
1879,1879,Cameron Ridge LLC,10586,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.3158,35.075&level=14",-118.315800,35.075000,06029006012,6.0,29.0,6012.0
2569,2569,Coram Energy LLC,54299,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.3417,35.075&level=14",-118.341700,35.075000,06029006012,6.0,29.0,6012.0
2570,2570,Coram Energy LLC (ECT),54298,CA,IPP Non-CHP,WT,WND,Onshore Wind Turbine,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-118.3417,35.0583&level=14",-118.341700,35.058300,06029006012,6.0,29.0,6012.0


In [60]:
df_with_geos['plant_name'].nunique()

12633

In [61]:
df_with_geos.to_csv('plants_with_geos.csv')

In [31]:
df_grants = pd.read_csv('US Environmental Protection Agency Environmental Justice Grants - Data.csv')

In [32]:
df_grants.head()

,Announcement Date,Award Date,Project Title,Project Description,Recipient,Funding Source,Type of Award,Funding Status,Federal Award Identification Number,Award Amount,...,Zip Code,City,County,State,Assistance Listing,Program,Investment Category,Website Url,Announcement Url,Data Extract Date
0,Date of announcement (for data on projects in ...,Date of award or rebate.,Descriptive title of the project.,Brief description of the project.,Name of recipient of the award.,Bipartisan Infrastructure Law (BIL) or Inflati...,Identifies if a grant is primary or sub-award.,Selected or Awarded.,Federal Award Identification Number (FAIN).,Funds (dollars) received from BIL or IRA.,...,Zip code of award/project place of performance.,City of award/project place of performance.,County of award/project place of performance.,State of award/project place of performance.,The Assistance Listing Number (formerly known ...,Bipartisan Infrastructure Law or Inflation Red...,"Investment category (Climate Action, Air Inves...",URL containing more detailed information about...,URL containing press release (announcing fundi...,Date data extracted.
1,NaN,05-31-2023,Morrisonville CUSD 1,With funding from the Bipartisan Infrastructur...,Morrisonville Cusd 1,BIL,Primary,Awarded,NaN,"790,000",...,NaN,NaN,NaN,IL,NaN,Clean School Bus Rebates,Air Investments,https://www.epa.gov/cleanschoolbus/awarded-cle...,https://www.epa.gov/cleanschoolbus/awarded-cle...,01-31-2025
2,NaN,01-07-2025,Saint Paul Island Renewable Energy Integration...,The purpose of this award is to provide fundin...,Aleut Community of Saint Paul Island,IRA,Primary,Awarded,84105401,"14,820,331",...,NaN,St Paul,Aleutians West Census Area,AK,66.046,CPRG - Implementation Grant,Climate Action,https://www.epa.gov/inflation-reduction-act/ab...,NaN,01-31-2025
3,NaN,07-10-2024,Cleanup Cooperative Agreement for City of West...,"Brownfields are real property, the expansion, ...",CITY OF WESTBROOK,BIL,Primary,Awarded,00A01199,"4,000,000",...,NaN,Westbrook,NaN,ME,66.818,Brownfields Projects,Land Investments,https://www.epa.gov/brownfields/bipartisan-inf...,NaN,01-31-2025
4,NaN,07-19-2023,Inflation Reduction Act &ndash; Climate Pollut...,The purpose of this grant agreement is to prov...,Sacramento Metropolitan AQMD,IRA,Primary,Awarded,98T74301,"1,000,000",...,NaN,NaN,Sacramento County,CA,66.046,CPRG - Planning Grant,Climate Action,https://www.epa.gov/inflation-reduction-act/ab...,NaN,01-31-2025


In [17]:
df_grants = df_grants.drop(0, axis=0).reset_index(drop=True)

In [18]:
df_grants.head()

,Announcement Date,Award Date,Project Title,Project Description,Recipient,Funding Source,Type of Award,Funding Status,Federal Award Identification Number,Award Amount,...,Zip Code,City,County,State,Assistance Listing,Program,Investment Category,Website Url,Announcement Url,Data Extract Date
0,NaN,05-31-2023,Morrisonville CUSD 1,With funding from the Bipartisan Infrastructur...,Morrisonville Cusd 1,BIL,Primary,Awarded,NaN,"790,000",...,NaN,NaN,NaN,IL,NaN,Clean School Bus Rebates,Air Investments,https://www.epa.gov/cleanschoolbus/awarded-cle...,https://www.epa.gov/cleanschoolbus/awarded-cle...,01-31-2025
1,NaN,01-07-2025,Saint Paul Island Renewable Energy Integration...,The purpose of this award is to provide fundin...,Aleut Community of Saint Paul Island,IRA,Primary,Awarded,84105401,"14,820,331",...,NaN,St Paul,Aleutians West Census Area,AK,66.046,CPRG - Implementation Grant,Climate Action,https://www.epa.gov/inflation-reduction-act/ab...,NaN,01-31-2025
2,NaN,07-10-2024,Cleanup Cooperative Agreement for City of West...,"Brownfields are real property, the expansion, ...",CITY OF WESTBROOK,BIL,Primary,Awarded,00A01199,"4,000,000",...,NaN,Westbrook,NaN,ME,66.818,Brownfields Projects,Land Investments,https://www.epa.gov/brownfields/bipartisan-inf...,NaN,01-31-2025
3,NaN,07-19-2023,Inflation Reduction Act &ndash; Climate Pollut...,The purpose of this grant agreement is to prov...,Sacramento Metropolitan AQMD,IRA,Primary,Awarded,98T74301,"1,000,000",...,NaN,NaN,Sacramento County,CA,66.046,CPRG - Planning Grant,Climate Action,https://www.epa.gov/inflation-reduction-act/ab...,NaN,01-31-2025
4,NaN,09-11-2023,Large Diameter Water Main Dead Ends Eliminatio...,This Bipartisan Infrastructure Law (BIL) (also...,DC Water and Sewer Authority,BIL,Primary,Awarded,95328501,"5,133,600",...,NaN,NaN,NaN,DC,66.468,Drinking Water State Revolving Fund,Water Investments,https://www.epa.gov/dwsrf,NaN,01-31-2025


In [33]:
import pandas as pd
import requests_cache
import time
import os
from tqdm import tqdm

# Enable caching to speed up repeated requests
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng, retries=3, delay=1):
    url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
    params = {
        "x": lng,
        "y": lat,
        "benchmark": "Public_AR_Census2020",
        "vintage": "Census2020_Census2020",
        "format": "json"
    }
    for attempt in range(retries):
        try:
            response = cache.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            tract = data['result']['geographies']['Census Tracts'][0]
            return tract
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(delay * (attempt + 1))  # Exponential backoff
            else:
                return {"error": str(e), "lat": lat, "lng": lng}

def process_chunk(chunk, chunk_index, save_prefix="census_geos_part"):
    results = []
    for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Processing chunk {chunk_index + 1}"):
        lat, lng = row['Latitude'], row['Longitude']
        result = geocode(lat, lng)
        results.append(result)

    # Convert the results into a DataFrame
    df_chunk = pd.DataFrame(results)
    output_path = f"{save_prefix}_{chunk_index + 1}.csv"
    df_chunk.to_csv(output_path, index=False)

    # Free memory by deleting intermediate data
    del results, df_chunk  
    return output_path

def geocode_in_chunks_safe(df, chunk_size=500, save_prefix="grants_geos_part"):
    total_chunks = (len(df) + chunk_size - 1) // chunk_size  # Calculate number of chunks
    
    for i in range(total_chunks):
        output_file = f"{save_prefix}_{i + 1}.csv"
        print(f"Processing chunk {i + 1} of {total_chunks}")
        
        # Skip checking if file exists; process each chunk regardless
        chunk = df.iloc[i*chunk_size : (i+1)*chunk_size]  # Select the chunk
        process_chunk(chunk, i, save_prefix=save_prefix)
        time.sleep(2)  # Sleep between requests to avoid hitting API rate limits

# Now, run this on your DataFrame
geocode_in_chunks_safe(df_grants)


Processing chunk 1 of 11


Processing chunk 1: 100%|█| 500/500 [00:03<00:00, 126.25it/s


Processing chunk 2 of 11


Processing chunk 2: 100%|█| 500/500 [00:00<00:00, 954.09it/s


Processing chunk 3 of 11


...

Processing chunk 9: 100%|█| 500/500 [00:00<00:00, 757.50it/s


Processing chunk 10 of 11


Processing chunk 10: 100%|█| 500/500 [00:00<00:00, 1069.83it


Processing chunk 11 of 11


Processing chunk 11: 100%|█| 226/226 [00:00<00:00, 906.61it/


In [36]:
import glob

# Adjust this pattern to match your saved files
chunk_files = sorted(glob.glob("grants_geos_part_*.csv"))

# Load and concatenate them
grants_df = pd.concat([pd.read_csv(f) for f in chunk_files], ignore_index=True)

# Optional: Save combined result to disk
grants_df.to_csv("grants_geocoded_full.csv", index=False)

# Check it out
grants_df.head()


,error,lat,lng,POP100,GEOID,CENTLAT,AREAWATER,STATE,BASENAME,OID,...,NAME,OBJECTID,TRACT,CENTLON,HU100,AREALAND,INTPTLON,MTFCC,UR,COUNTY
0,400 Client Error: for url: https://geocoding....,Latitude of award/project place of performance.,Longitude of award/project place of performance.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,4796.0,1.702196e+10,39.441237,4.022030e+06,17.0,9590.00,2.079015e+13,...,Census Tract 9590,83511.0,959000.0,-89.387215,2119.0,4.910069e+08,-89.388553,G5020,M,21.0
2,NaN,NaN,NaN,978.0,2.016000e+09,52.593343,2.207908e+10,2.0,1.00,2.079045e+12,...,Census Tract 1,17011.0,100.0,-176.454970,555.0,8.556804e+09,178.338813,G5020,R,16.0
3,NaN,NaN,NaN,5175.0,2.300500e+10,43.710389,1.286170e+05,23.0,27.00,2.079020e+13,...,Census Tract 27,48443.0,2700.0,-70.362032,2289.0,1.271936e+07,-70.365230,G5020,M,5.0
4,NaN,NaN,NaN,1609.0,6.067009e+09,38.438152,0.000000e+00,6.0,93.32,2.079037e+14,...,Census Tract 93.32,66858.0,9332.0,-121.339287,498.0,7.870005e+06,-121.339287,G5020,M,67.0


In [37]:
grants_df['GEOID'] = grants_df['GEOID'].apply(
    lambda x: str(int(float(x))).zfill(11) if pd.notnull(x) else x
)

In [38]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT']
grants_geos_df = grants_df[to_keep]
grants_geos_df

,GEOID,STATE,COUNTY,TRACT
0,NaN,NaN,NaN,NaN
1,17021959000,17.0,21.0,959000.0
2,02016000100,2.0,16.0,100.0
3,23005002700,23.0,5.0,2700.0
4,06067009332,6.0,67.0,9332.0
...,...,...,...,...
5221,04001944301,4.0,1.0,944301.0
5222,72107955001,72.0,107.0,955001.0
5223,17201004004,17.0,201.0,4004.0
5224,27035951100,27.0,35.0,951100.0


In [39]:
grants_with_geos = pd.concat(
    [ 
        df_grants.reset_index(drop=True),
        grants_geos_df.reset_index(drop=True)
    ], 
    axis=1)

grants_with_geos.head()

,Announcement Date,Award Date,Project Title,Project Description,Recipient,Funding Source,Type of Award,Funding Status,Federal Award Identification Number,Award Amount,...,Assistance Listing,Program,Investment Category,Website Url,Announcement Url,Data Extract Date,GEOID,STATE,COUNTY,TRACT
0,Date of announcement (for data on projects in ...,Date of award or rebate.,Descriptive title of the project.,Brief description of the project.,Name of recipient of the award.,Bipartisan Infrastructure Law (BIL) or Inflati...,Identifies if a grant is primary or sub-award.,Selected or Awarded.,Federal Award Identification Number (FAIN).,Funds (dollars) received from BIL or IRA.,...,The Assistance Listing Number (formerly known ...,Bipartisan Infrastructure Law or Inflation Red...,"Investment category (Climate Action, Air Inves...",URL containing more detailed information about...,URL containing press release (announcing fundi...,Date data extracted.,NaN,NaN,NaN,NaN
1,NaN,05-31-2023,Morrisonville CUSD 1,With funding from the Bipartisan Infrastructur...,Morrisonville Cusd 1,BIL,Primary,Awarded,NaN,"790,000",...,NaN,Clean School Bus Rebates,Air Investments,https://www.epa.gov/cleanschoolbus/awarded-cle...,https://www.epa.gov/cleanschoolbus/awarded-cle...,01-31-2025,17021959000,17.0,21.0,959000.0
2,NaN,01-07-2025,Saint Paul Island Renewable Energy Integration...,The purpose of this award is to provide fundin...,Aleut Community of Saint Paul Island,IRA,Primary,Awarded,84105401,"14,820,331",...,66.046,CPRG - Implementation Grant,Climate Action,https://www.epa.gov/inflation-reduction-act/ab...,NaN,01-31-2025,02016000100,2.0,16.0,100.0
3,NaN,07-10-2024,Cleanup Cooperative Agreement for City of West...,"Brownfields are real property, the expansion, ...",CITY OF WESTBROOK,BIL,Primary,Awarded,00A01199,"4,000,000",...,66.818,Brownfields Projects,Land Investments,https://www.epa.gov/brownfields/bipartisan-inf...,NaN,01-31-2025,23005002700,23.0,5.0,2700.0
4,NaN,07-19-2023,Inflation Reduction Act &ndash; Climate Pollut...,The purpose of this grant agreement is to prov...,Sacramento Metropolitan AQMD,IRA,Primary,Awarded,98T74301,"1,000,000",...,66.046,CPRG - Planning Grant,Climate Action,https://www.epa.gov/inflation-reduction-act/ab...,NaN,01-31-2025,06067009332,6.0,67.0,9332.0


In [40]:
grants_with_geos.to_csv('grants_with_geos.csv')

In [64]:
df_proposed = pd.read_csv('maryland_possible_locations.csv')

In [66]:
df_proposed.head()

,County,District,Power Plant,Owner,"Description (note: current gas sites may not be eligiblefor new gas if there is no GHG reduction, but could become nuclear sites)",Latitude,Longitude
0,ALLEGHENY,1C,Warrior Run,AES Warrior Run Ltd. Partnership,"180 MW, coal fired steam boiler, closed June o...",39.597398,-78.744465
1,ALLEGHENY,1A,Luke Mill,New Page Corp,"~60 MW, 2 coal fired steam boilers, Closed 2019",39.474072,-79.055827
2,ANNE ARUNDEL,31,Brandon Shores,Talen Energy (Raven Power Holdings LLC),"1,273 MW, 2 units, coal and steam boilers, sch...",39.179750,-76.536705
3,ANNE ARUNDEL,31,Herbert Wagner,Talen Energy (Raven Power Holdings LLC),"975.9 MW total (Unit 1 – Gas, Unit 2 - coal 13...",39.178669,-76.526322
4,BALTIMORE CITY,7A,C.P. Crane,Avenue Capital Group,"400 MW, coal; closed 2018, demolished 2022, un...",39.325291,-76.366534


In [69]:
import pandas as pd
import requests_cache
import time
from tqdm import tqdm

# Enable caching to speed up repeated requests
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng, retries=3, delay=1):
    url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
    params = {
        "x": lng,
        "y": lat,
        "benchmark": "Public_AR_Census2020",
        "vintage": "Census2020_Census2020",
        "format": "json"
    }
    for attempt in range(retries):
        try:
            response = cache.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            tract = data['result']['geographies']['Census Tracts'][0]
            return tract
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(delay * (attempt + 1))  # Exponential backoff
            else:
                return {"error": str(e), "lat": lat, "lng": lng}

def geocode_all(df_proposed, save_path="census_geos_all.csv"):
    results = []
    for _, row in tqdm(df_proposed.iterrows(), total=len(df_proposed), desc="Processing all data"):
        lat, lng = row['Latitude'], row['Longitude']
        result = geocode(lat, lng)
        results.append(result)

    # Convert the results into a DataFrame
    df_results = pd.DataFrame(results)
    df_results.to_csv(save_path, index=False)

    # Free memory by deleting intermediate data
    del results, df_results  
    print(f"Results saved to {save_path}")

# Now, run this on your DataFrame
geocode_all(df_proposed)

Processing all data: 100%|██████████████████████| 34/34 [00:15<00:00,  2.22it/s]

Results saved to census_geos_all.csv


In [70]:
census_geos = pd.read_csv('census_geos_all.csv')

In [71]:
census_geos.head()

,POP100,GEOID,CENTLAT,AREAWATER,STATE,BASENAME,OID,LSADC,FUNCSTAT,INTPTLAT,NAME,OBJECTID,TRACT,CENTLON,HU100,AREALAND,INTPTLON,MTFCC,UR,COUNTY
0,4097,24001000200,39.625722,3080200,24,2.00,20790762624028,CT,S,39.612313,Census Tract 2,27367,200,-78.687833,1425,124505956,-78.703104,G5020,M,1
1,3684,24001002200,39.486235,1043712,24,22.00,20790762624213,CT,S,39.498072,Census Tract 22,27031,2200,-78.994125,1778,60150344,-78.985812,G5020,M,1
2,12283,24003730102,39.179930,4203009,24,7301.02,2079015484997092,CT,S,39.180121,Census Tract 7301.02,29315,730102,-76.552462,4813,19456267,-76.556988,G5020,U,3
3,12283,24003730102,39.179930,4203009,24,7301.02,2079015484997092,CT,S,39.180121,Census Tract 7301.02,29315,730102,-76.552462,4813,19456267,-76.556988,G5020,U,3
4,4653,24005451803,39.328623,1168179,24,4518.03,20790271414184,CT,S,39.325194,Census Tract 4518.03,55950,451803,-76.377431,1998,3662472,-76.372650,G5020,M,5


In [72]:
census_geos['GEOID'] = census_geos['GEOID'].apply(lambda x: f"{int(x):011d}" if pd.notna(x) else x)

In [73]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT']
census_geos_df = census_geos[to_keep]
census_geos_df

,GEOID,STATE,COUNTY,TRACT
0,24001000200,24,1,200
1,24001002200,24,1,2200
2,24003730102,24,3,730102
3,24003730102,24,3,730102
4,24005451803,24,5,451803
5,24510240100,24,510,240100
6,24005492600,24,5,492600
7,24510250301,24,510,250301
8,24510260404,24,510,260404
9,24510240400,24,510,240400


In [74]:
df_proposed_with_geos = pd.concat(
    [ 
        df_proposed.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

df_proposed_with_geos.head()

,County,District,Power Plant,Owner,"Description (note: current gas sites may not be eligiblefor new gas if there is no GHG reduction, but could become nuclear sites)",Latitude,Longitude,GEOID,STATE,COUNTY,TRACT
0,ALLEGHENY,1C,Warrior Run,AES Warrior Run Ltd. Partnership,"180 MW, coal fired steam boiler, closed June o...",39.597398,-78.744465,24001000200,24,1,200
1,ALLEGHENY,1A,Luke Mill,New Page Corp,"~60 MW, 2 coal fired steam boilers, Closed 2019",39.474072,-79.055827,24001002200,24,1,2200
2,ANNE ARUNDEL,31,Brandon Shores,Talen Energy (Raven Power Holdings LLC),"1,273 MW, 2 units, coal and steam boilers, sch...",39.179750,-76.536705,24003730102,24,3,730102
3,ANNE ARUNDEL,31,Herbert Wagner,Talen Energy (Raven Power Holdings LLC),"975.9 MW total (Unit 1 – Gas, Unit 2 - coal 13...",39.178669,-76.526322,24003730102,24,3,730102
4,BALTIMORE CITY,7A,C.P. Crane,Avenue Capital Group,"400 MW, coal; closed 2018, demolished 2022, un...",39.325291,-76.366534,24005451803,24,5,451803


In [75]:
df_proposed_with_geos.to_csv('maryland_possible_locations_geo.csv',index=False)

### Step 3 | Get Census Geographies

In [ ]:
# Code adapted from:
# https://gis.stackexchange.com/questions/363830/applying-the-censusgeocode-package-to-an-entire-dataframe-of-geocoded-data
# Defines a geocode function that accepts lat/long and spits out geographies
# The code then runs that funciton in parllel (for speed).

import pandas as pd
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Tracts'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        latitudes = df['latitude']
        longitudes = df['longitude']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        mapped_results = [result for result in mapped_results if result is not None]
        data = list(tqdm(mapped_results, total=len(mapped_results)))  # Use filtered results

    return pd.DataFrame(data)

census_geos_df = bulk_geocode(df['latitude'], df['longitude']) 
census_geos_df.head()

Error geocoding (29.449827, -101.06011): 'Census Tracts'


In [ ]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT']
census_geos_df = census_geos_df[to_keep]
census_geos_df

In [ ]:
df_with_geos = pd.concat(
    [ 
        df.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

df_with_geos.head()

In [ ]:
df_with_geos = df_with_geos.drop(columns=["Unnamed: 0"])

In [ ]:
df_with_geos.to_csv('plants_with_geos.csv', index=False)

# Step 4 | Pick a geographical level and get Census data
Do you want Census data at the state level? county? tract? block?

1. Pick a geographical level.
2. See `census-example.ipynb` if you want to learn how to get Census data at your desired level

# Hope that helps!